In [1]:
!pip install gensim


[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
import re
from gensim.models import Word2Vec
from collections import Counter

In [3]:
current_dir = os.getcwd()
BASE_PATH = os.path.join(current_dir, "ilur-news-corpus", "ilur-news-corpus", "train")
categories = ['accidents', 'culture', 'economy', 'politics', 'society', 'sport', 'weather']

# Models's parameters
CONFIG = {
    'vector_size': 300,
    'window': 5,
    'negative': 15,
    'epochs': 20,
    'min_count': 5,      # հեռացնել հազվադեպ բառերը
    'sg': 1,             # Skip-gram architecture
    'workers': 4
}

print(" Model parameter:")
for key, value in CONFIG.items():
    print(f"   {key}: {value}")

 Model parameter:
   vector_size: 300
   window: 5
   negative: 15
   epochs: 20
   min_count: 5
   sg: 1
   workers: 4


In [4]:
#  Data Preprocessing Functions

def clean_armenian_text(text):

    # 1. Lowercase
    text = text.lower()

    # 2. delete dates (25.11.2024, 25/11/2024)
    text = re.sub(r'\d{1,2}[./]\d{1,2}[./]\d{2,4}', '', text)

    # 3. delete times (14:30, 14:30:45)
    text = re.sub(r'\d{1,2}:\d{2}(:\d{2})?', '', text)

    # 4. delete digits
    text = re.sub(r'\d+', '', text)

    #Unicode: \u0531-\u0587
    text = re.sub(r'[^\u0531-\u0587\s.!?]', ' ', text)

    # 6. Normalize spaces
    text = re.sub(r'\s+', ' ', text)

    # 7. Trim
    text = text.strip()

    return text

def split_into_sentences(text):

    #  . ! ?
    sentences = re.split(r'[.!?]+', text)
    sentences = [s.strip() for s in sentences if len(s.strip()) > 0]

    return sentences


def tokenize_sentence(sentence):

    words = sentence.split()
    return words if len(words) >= 2 else []


In [5]:
#Load և Process All Data

def load_and_process_all_data(base_path, categories):

    all_sentences = []
    all_words = []

    stats = {
        'files_processed': 0,
        'files_failed': 0,
        'total_sentences': 0,
        'total_words': 0
    }
    print("\n Reading and processing all data...")
    print("="*60)

    for category in categories:
        category_path = os.path.join(base_path, category)
        category_files = 0

        if not os.path.exists(category_path):
            print(f"⚠  {category}: folder not found")
            continue

        for filename in os.listdir(category_path):
            if not filename.endswith('.txt'):
                continue

            file_path = os.path.join(category_path, filename)

            try:
                # read file
                with open(file_path, 'r', encoding='utf-8') as f:
                    text = f.read()
                    # text cleaning
                cleaned_text = clean_armenian_text(text)

                if len(cleaned_text) < 10:  # Skip
                    continue

                sentences = split_into_sentences(cleaned_text)

                # Tokenize sentences
                for sentence in sentences:
                    words = tokenize_sentence(sentence)
                    if words:
                        all_sentences.append(words)
                        all_words.extend(words)
                        stats['total_sentences'] += 1
                        stats['total_words'] += len(words)

                stats['files_processed'] += 1
                category_files += 1

                # Progress
                if stats['files_processed'] % 100 == 0:
                    print(f"   Processed: {stats['files_processed']} files...")

            except Exception as e:
                stats['files_failed'] += 1
                print(f" Error in {filename}: {e}")

        print(f"✓  {category:12s}: {category_files} files")

    print("="*60)
    print(f" Finish !\n")

    return all_sentences, all_words, stats

# Load data
sentences, all_words, stats = load_and_process_all_data(BASE_PATH, categories)



 Reading and processing all data...
   Processed: 100 files...
   Processed: 200 files...
   Processed: 300 files...
   Processed: 400 files...
   Processed: 500 files...
   Processed: 600 files...
   Processed: 700 files...
   Processed: 800 files...
   Processed: 900 files...
   Processed: 1000 files...
   Processed: 1100 files...
✓  accidents   : 1163 files
   Processed: 1200 files...
   Processed: 1300 files...
   Processed: 1400 files...
   Processed: 1500 files...
   Processed: 1600 files...
   Processed: 1700 files...
   Processed: 1800 files...
   Processed: 1900 files...
✓  culture     : 796 files
   Processed: 2000 files...
   Processed: 2100 files...
   Processed: 2200 files...
   Processed: 2300 files...
   Processed: 2400 files...
   Processed: 2500 files...
   Processed: 2600 files...
   Processed: 2700 files...
   Processed: 2800 files...
   Processed: 2900 files...
   Processed: 3000 files...
   Processed: 3100 files...
   Processed: 3200 files...
   Processed: 3300 fi

In [6]:
print("ՎԻՃԱԿԱԳՐՈՒԹՅՈՒՆ:")
print("="*60)
print(f"   Ֆայլեր (processed):      {stats['files_processed']}")
print(f"   Ֆայլեր (failed):         {stats['files_failed']}")
print(f"   Նախադասություններ:       {stats['total_sentences']:,}")
print(f"   Ընդամենը բառեր:         {stats['total_words']:,}")

word_freq = Counter(all_words)
unique_words = len(word_freq)

min_count = CONFIG['min_count']
rare_words = sum(1 for count in word_freq.values() if count < min_count)
kept_words = unique_words - rare_words

print(f"\n   Unique բառեր:           {unique_words:,}")
print(f"   Հազվադեպ բառեր (< {min_count}):  {rare_words:,}")
print(f"   Կպահվող բառեր:          {kept_words:,}")

print(f"\n Ամենահաճախ 15 բառերը:")
for word, count in word_freq.most_common(15):
    print(f"   {word:15s}: {count:6,}")

print("="*60)

ՎԻՃԱԿԱԳՐՈՒԹՅՈՒՆ:
   Ֆայլեր (processed):      9941
   Ֆայլեր (failed):         0
   Նախադասություններ:       34,793
   Ընդամենը բառեր:         1,389,413

   Unique բառեր:           94,983
   Հազվադեպ բառեր (< 5):  72,356
   Կպահվող բառեր:          22,627

 Ամենահաճախ 15 բառերը:
   է              : 63,859
   և              : 21,432
   որ             : 18,232
   են             : 17,826
   ի              : 11,745
   ին             : 10,964
   եւ             :  8,073
   էր             :  6,483
   համար          :  6,223
   ու             :  5,909
   այս            :  5,545
   հետ            :  5,499
   այդ            :  5,144
   հհ             :  4,717
   իսկ            :  4,711


In [7]:
# Train Word2Vec Model

print("\n training Word2Vec model...")
print("="*60)

model = Word2Vec(
    sentences=sentences,
    vector_size=CONFIG['vector_size'],
    window=CONFIG['window'],
    min_count=CONFIG['min_count'],
    workers=CONFIG['workers'],
    sg=CONFIG['sg'],
    negative=CONFIG['negative'],
    epochs=CONFIG['epochs']
)

print(" Training is finished")
print(f"   Vocabulary size: {len(model.wv):,} word")
print("="*60)


 training Word2Vec model...
 Training is finished
   Vocabulary size: 22,627 word


In [8]:
# Save Model

model.save("armenian_word2vec.model")
model.wv.save("armenian_word_vectors.kv")
print(f"\n Saved:")
print(f"   - armenian_word2vec.model")
print(f"   - armenian_word_vectors.kv")



 Saved:
   - armenian_word2vec.model
   - armenian_word_vectors.kv


In [9]:
# Model Evaluation

print("\n🔍 Model Evaluation:")
print("="*60)

print("\n1️⃣ Similar words:\n")

test_words = ['երևան', 'հայաստան', 'կառավարություն', 'սպորտ', 'եղանակ']

for word in test_words:
    if word in model.wv:
        similar = model.wv.most_similar(word, topn=5)
        print(f"   '{word}' →")
        for sim_word, score in similar:
            print(f"      {sim_word:20s} ({score:.3f})")
        print()
    else:
        print(f"   ⚠️ '{word}' not found in vocabulary\n")

print("="*60)
print("✅ ")


🔍 Model Evaluation:

1️⃣ Similar words:

   'երևան' →
      քաղաքում՝            (0.528)
      երևանում՝            (0.514)
      երևանում՝մարտի       (0.510)
      քաղաքում՝դեկտեմբերի  (0.506)
      քաղաքի               (0.506)

   'հայաստան' →
      բարգավաճ             (0.457)
      իրան                 (0.446)
      լյուքսեմբուրգ        (0.436)
      վրաստան              (0.435)
      հանրապետություն      (0.433)

   'կառավարություն' →
      արվելու              (0.457)
      իշխանություն         (0.437)
      հեռանկար             (0.436)
      հասարակություն       (0.423)
      հարստանում           (0.420)

   'սպորտ' →
      էքսպրես              (0.638)
      դահուկային           (0.526)
      աթլետիկա             (0.492)
      շաբաթաթերթի          (0.455)
      սուպերմիջին          (0.450)

   'եղանակ' →
      տեղումների           (0.761)
      անձրև                (0.673)
      սպասվում             (0.647)
      ամպրոպ               (0.628)
      մառախուղից           (0.619)

